In [5]:
pip install pandas matplotlib seaborn numpy scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 74.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 67.7 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 38.6 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 57.0 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 77.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 79.9 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 41.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 51.9 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 80.6 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.2 MB/s  0:00:0

In [12]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
%matplotlib inline
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
#%pip install torch torchvision
# You might want to install the gpu version when working on larger datasets in computer vision or NLP domains.
import torch
import torch.nn as nn #this submodule provides tools for building and training neural networks
import torch.optim as optim #this submodule contains information on optimizers such as SGD
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons
from torch.utils.data import DataLoader, TensorDataset

In [9]:
housing = pd.read_csv("housing.csv")
X = housing.copy().drop(["ocean_proximity"], axis=1)
Y = housing.copy()["ocean_proximity"]
Y = housing.copy()["ocean_proximity"].map({"<1H OCEAN":0, "INLAND":1,
                                        "ISLAND": 2, "NEAR BAY": 3,
                                        "NEAR OCEAN": 4}).values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,
                                                    train_size=0.8, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train,
                                                  test_size=0.2, train_size=0.8,
                                                  random_state=42)

In [15]:
torch.cuda.get_device_name(1)

'NVIDIA L40S'

In [11]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X_train)

X_train = imputer.transform(X_train)
X_dev = imputer.transform(X_dev)
X_test = imputer.transform(X_test)

scaler = preprocessing.StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_dev = scaler.transform(X_dev)
X_test = scaler.transform(X_test)

In [19]:
# Create DataLoader for batching
train_data = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_data = TensorDataset(torch.tensor(X_dev), torch.tensor(y_dev))
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
def create_model(hidden_size, learning_rate, shape, optimi):
    l = []
    for _ in range(hidden_size):
        
        l.append(nn.Linear(shape, shape))
        l.append(nn.ReLU())
    m = nn.Sequential(
        nn.Linear(9, shape),
        *l,
        nn.Linear(shape, 5)
    )
    if optimi == 'sgd':
        optimizer = optim.SGD(m.parameters(), lr=learning_rate)
    elif optimi == 'adam':
        optimizer = optim.Adam(m.parameters(), lr=learning_rate) 
    return m, optimizer

In [25]:
def train_and_val(model, optimizer, num_epochs=10, train_loader=train_loader, val_loader=val_loader):
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Training step
        for inputs, labels in train_loader:
            inputs = inputs.float().to(device)
            labels = labels.long().to(device)
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)  # Output is a single logit per sample
            train_loss = criterion(outputs,labels)
            train_loss.backward()
            optimizer.step()
            
            running_loss += train_loss.item()
            predicted = torch.softmax(outputs, dim=1) # Apply softmax to logits and classify
            predicted = torch.argmax(predicted, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_epoch_loss = running_loss / len(train_loader)
        train_accuracy = correct / total
        
        # Validation step
        model.eval()  # Set the model to evaluation mode
        val_running = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():  # Disable gradient computation for validation
            for inputs, labels in val_loader:
                inputs = inputs.float().to(device)
                labels = labels.long().to(device)
                outputs = model(inputs)
                val_loss = criterion(outputs,labels)
                val_running += val_loss.item()
                predicted = torch.softmax(outputs, dim=1) # Apply softmax to logits and classify
                predicted = torch.argmax(predicted, dim=1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
                    
        val_epoch_loss = val_running / len(val_loader)
        val_accuracy = val_correct / val_total
                

        # Record the losses
        # history['epoch'].append(epoch + 1)
        # history['train_loss'].append(train_loss.item())
        # history['val_loss'].append(val_loss.item())


        # Print losses every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], "
                f"Train Loss: {train_loss.item():.4f}, "
                f"Train Accuracy: {train_accuracy:.4f}, "
                f"Validation Loss: {val_loss.item():.4f}, "
                f"Validation Accuracy: {val_accuracy:.4f}")

    # Print final losses after training
    print(f"Final Train Loss: {train_loss:.4f}")
    print(f"Final Validation Loss: {val_loss:.4f}")
    print(f"Final Validation Accuracy: {val_accuracy:.4f}")
    
    return val_accuracy
    
    

In [26]:
torch.manual_seed(42)  # PyTorch random seed

In [28]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tqdm import tqdm
hidden_sizes = [1,2,3]
shape = [16,32,64]
learning_rates = [0.01, 0.001]
optimizers = ['sgd', 'adam']
models = pd.DataFrame()
total_iterations = len(hidden_sizes) * len(shape) * len(learning_rates) * len(optimizers)

with tqdm(total=total_iterations, desc="Training Models") as pbar:
    for a in hidden_sizes:
        for b in shape:
            for c in learning_rates:
                for d in optimizers:
                    model, optimizer = create_model(a, c, b, d)
                    model.to(device)
                    m_name = f'model_h{a}_s{b}_lr{c}_{d}'
                    print(f"Running {m_name}")
                    models[m_name] = train_and_val(model, optimizer)

Training Models:   0%|          | 0/36 [00:00<?, ?it/s]

Running model_h1_s16_lr0.01_sgd
Epoch [10/10], Train Loss: 0.4984, Train Accuracy: 0.8016, Validation Loss: 0.4565, Validation Accuracy: 0.7962
Final Train Loss: 0.4984
Final Validation Loss: 0.4565
Final Validation Accuracy: 0.7962
Running model_h1_s16_lr0.01_adam
Epoch [10/10], Train Loss: 0.4329, Train Accuracy: 0.8853, Validation Loss: 0.1501, Validation Accuracy: 0.8771
Final Train Loss: 0.4329
Final Validation Loss: 0.1501
Final Validation Accuracy: 0.8771
Running model_h1_s16_lr0.001_sgd
Epoch [10/10], Train Loss: 0.9781, Train Accuracy: 0.6116, Validation Loss: 0.9643, Validation Accuracy: 0.6097
Final Train Loss: 0.9781
Final Validation Loss: 0.9643
Final Validation Accuracy: 0.6097
Running model_h1_s16_lr0.001_adam
Epoch [10/10], Train Loss: 0.3486, Train Accuracy: 0.8657, Validation Loss: 0.3888, Validation Accuracy: 0.8725
Final Train Loss: 0.3486
Final Validation Loss: 0.3888
Final Validation Accuracy: 0.8725
Running model_h1_s32_lr0.01_sgd
Epoch [10/10], Train Loss: 0.439